I passaggi da effettuare sono
- Corrispondenza punti 2D e 3D
- Trovare la matrice P
- Usare fattorizzazione RQ (Variante della fattorizzazione QR)
- Trovare K che è la matrice R della fattorizzazione precedente
- Mentre la matrice R di rotazione (da non confondere con la R di RQ) sarà la matrice Q di RQ
- Trovare t vettore colonna della traslazione

Import di tutte le librerie necessarie
Usiamo numpy per la gestione delle matrici generate dalle immagini e math per fare i calcoli 

In [ ]:
import numpy
import math
import cv2

Iniziamo mappando i punti 3D nello spazio 2D attraverso dei vettori di punti.
Questi punti sono stati definiti dalla professoressa

In [ ]:
def define_Points():
    n_corners = 14

    #3D points
    objp = np.zeros((n_corners,3), np.float32)

    #first face
    objp[1,:] =[7, 0, 0]
    objp[2,:] =[7, 0, 7]
    objp[3,:] =[0, 0, 7]
    objp[4,:] =[3, 0, 4]

    #second face
    objp[5,:] =[0, 7, 0]
    objp[6,:] =[0, 7, 7]
    objp[7,:] =[0, 4, 4]

    #top face
    objp[8,:] =[7, 7, 7]
    objp[9,:] =[4, 4, 7]

    #additional points
    objp[10,:] =[7, 0, 3]
    objp[11,:] =[0, 3, 7]
    objp[12,:] =[0, 7, 3]
    objp[13,:] =[7, 2, 7]

    #2D points
    imgp = np.zeros((n_corners,2), np.float32)

    imgp[0,:] = [103, 254]
    imgp[1,:] = [252, 212]
    imgp[2,:] = [258, 39]
    imgp[3,:] = [103, 61]
    imgp[4,:] = [173, 132]

    imgp[5,:] = [18, 196]
    imgp[6,:] = [14, 26]
    imgp[7,:] = [50, 120]

    imgp[8,:] = [155, 10]
    imgp[9,:] = [136, 29]

    #additional points
    imgp[10,:] = [254, 141]
    imgp[11,:] = [62, 46]
    imgp[12,:] = [16, 127]
    imgp[13,:] = [224, 27]
    
    return objp, imgp


Definiziamo poi una funzione che stampa sull'immagine i punti che abbiamo definito prima nella funzione define_points()

In [ ]:
def show_points(imgp):
    # Let's show the selected points 
    img = cv2.imread("image_to_cal.png")
    img_to_show = img.copy()
    for p in imgp:
        img_to_show = cv2.circle(img_to_show, p.astype(np.int32), 3, (255, 0, 0) )

    cv2.imshow("img", img_to_show)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

Carichiamo quindi l'immagine e stampiamo tutto a schermo

In [ ]:
img = cv2.imread("image_to_cal.png")
objp, imgp = define_Points()
show_points(imgp)

La variabile objp avrà quindi all'interno le coordinate nello spazio in una matrice 14,3 e che corrisponde alla matrice che contiene i punti tridimensionali e imgp è la matrice che contiene gli stessi punti ma nello spazio 2D dell'immagine

Iniziamo quindi i calcoli per trovare la matrice P